# Notebook to experiment with the MT5 TSystems' abstractive summarizer model

Author: Hadi, Ver 202304

I am interested in two things: Speed and Quality. Quality itself has three aspects, (i) not fibbing (ii) not cutting off (iii) sufficient info

In [1]:
# LOAD A WEBPAGE FOR ANALYSIS. Extracts pages content as text into `txt` variable

import requests
import html2text  # pip
import lxml.html

#url = "https://www.dw.com/en/brazil-welcomes-russias-lavrov-amid-us-criticism/a-65353778"
url = "https://www.dw.com/en/apple-opens-first-flagship-store-in-india/a-65354938"

response = requests.get(url)

tree = lxml.html.fromstring(response.text)
el = tree.xpath('//main|//div[@role="main"]|//section[@role="main"]|//div[@class="main-row"]|//div[@class="main"]|//div[@id="main"]')
if el:
    src = "\n".join(el.getall())
    print("found")
else:
    src = response.text
    #print("not found")

h = html2text.HTML2Text()
h.ignore_links = True
h.ignore_emphasis = True
h.images_to_alt = False
h.single_line_break = True
txt = h.handle(src)

# let's remove the fluf manually on this page
if "# Brazil" in txt:
    txt = txt[txt.find("# Brazil welcomes Russia's"):txt.find("mf/wd (AP, AFP)")]
elif "# Apple" in txt:
    txt = txt[txt.find("# Apple opens first"):txt.find("mf/nm (AP, Reuters)")]
print(txt)

# Apple opens first flagship store in India

22 hours ago22 hours ago

Hundreds of people waited in line outside the new Apple store in the financial
hub Mumbai. The company is planning to boost its production in India — the
second-largest smartphone market in the world.

https://p.dw.com/p/4QDnu

Advertisement

Apple CEO Tim Cook was in Mumbai on Tuesday to inaugurate the company's first
retail store in India.

"India has such a beautiful culture and an incredible energy, and we're
excited to build on our long-standing history," Cook said in a statement.

Bloggers, tech analysts and Bollywood celebrities were invited to the opening
of the new store in Mumbai's Jio World Drive shopping mall.

About 300 Apple fans were seen lining up outside the store, some of them
waiting since the previous night.

"The fanboy inside me would not listen," 30-year-old Purav Mehta, who wanted
to get Cook's autograph, told Reuters news agency.

> A second store is set to open in the capital, New Delhi, on

In [2]:
# TODO:  use a method to detect the text language, also fast

print(len(txt), len(txt.split("\n")))  # 3121 chars, 80 lines, xx tokens

1991 52


In [3]:
from transformers import MT5ForConditionalGeneration, MT5Tokenizer
%time tokenizer = MT5Tokenizer.from_pretrained("T-Systems-onsite/mt5-small-sum-de-en-v2")  # 0.5s, once
%time model = MT5ForConditionalGeneration.from_pretrained("T-Systems-onsite/mt5-small-sum-de-en-v2")  # 4.3s, once
#model.half()  # 92ms, speeds up inference, mem at 708MiB
model.cuda()  # 99ms
# model.eval() unnecessary

!nvidia-smi

CPU times: user 379 ms, sys: 7.54 ms, total: 387 ms
Wall time: 547 ms
CPU times: user 3.86 s, sys: 629 ms, total: 4.49 s
Wall time: 3.34 s


RuntimeError: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero.

In [49]:
# TOKENIZER RELATED QUESTIONS/INVESTIGATIONS.
# - use a fast tokenizer to get number of tokens (should be >20tokz, <512tokz) → resolved, below
# - actualy best method is to use the tokenizer from the models themselves

# import spacy
# SPACY_NLP = spacy.load("de_core_news_sm")
# s = list(SPACY_NLP(txt).sents)[0]
# print(len(s), len(str(s)))
#
# from transformers import AutoTokenizer
# tokz = AutoTokenizer.from_pretrained(('symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli'))
# print(tokz.model_max_length)

32 201


512


In [4]:
# mmm, max_new_tokens=20 does good job. without it not so much.

input_ids = tokenizer.encode(txt, return_tensors="pt", max_length=512, truncation=True)  # 3ms

# 625ms on GPU with or without beams
%time output = model.generate(input_ids=input_ids.cuda(), max_new_tokens=120, repetition_penalty=1.5)  # do_sample=True, num_beams=4)

print(tokenizer.decode(output[0], skip_special_tokens=True))

RuntimeError: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero.

NameError: name 'output' is not defined

In [6]:
# INTEL int8 quantization
#!pip install onnx neural-compressor git+https://github.com/huggingface/optimum-intel.git

from optimum.intel import INCQuantizer
from optimum.intel import INCModelForSeq2SeqLM
from neural_compressor.config import PostTrainingQuantConfig

# Load model to quantize. Keep it on cpu and FP32.
from transformers import AutoTokenizer, AutoModel
%time model = MT5ForConditionalGeneration.from_pretrained("T-Systems-onsite/mt5-small-sum-de-en-v2")  # 4.3s
tokenizer = AutoTokenizer.from_pretrained("T-Systems-onsite/mt5-small-sum-de-en-v2")  # 1s, once

# Apply dynamic quantization and save the resulting model
quantizer = INCQuantizer.from_pretrained(model)
%time quantizer.quantize(quantization_config=PostTrainingQuantConfig(approach="dynamic"), save_directory="iquant_mt5sum")  # 20s


# lets reload it
%time model_iq = INCModelForSeq2SeqLM.from_pretrained("iquant_mt5sum")  # 15-25s!


/home/hadi/.venv/simba2/lib/python3.10/site-packages/optimum/intel/neural_compressor/quantization.py:657: UserWarning: The class `IncQuantizer` has been depreciated and will be removed in optimum-intel v1.7, please use `INCQuantizer` instead.
  warnings.warn(
/home/hadi/.venv/simba2/lib/python3.10/site-packages/optimum/intel/neural_compressor/trainer.py:817: UserWarning: The class `IncTrainer` has been depreciated and will be removed in optimum-intel v1.7, please use `INCTrainer` instead.
  warnings.warn(
2023-04-19 09:33:12 [WARNING] Force convert framework model to neural_compressor model.
2023-04-19 09:33:12 [INFO] Start auto tuning.


CPU times: user 4.06 s, sys: 473 ms, total: 4.53 s
Wall time: 3.32 s


2023-04-19 09:33:12 [INFO] Pass query framework capability elapsed time: 766.73 ms
2023-04-19 09:33:12 [INFO] Adaptor has 1 recipes.
2023-04-19 09:33:12 [INFO] 0 recipes specified by user.
2023-04-19 09:33:12 [INFO] 0 recipes require future tuning.
2023-04-19 09:33:12 [INFO] *** Initialize auto tuning
2023-04-19 09:33:12 [INFO] Get FP32 model baseline.
2023-04-19 09:33:12 [INFO] Save tuning history to /home/hadi/Source/simba/summ-model/nc_workspace/2023-04-19_09-31-26/./history.snapshot.
2023-04-19 09:33:12 [INFO] FP32 baseline is: [Accuracy: 1.0000, Duration (seconds): 0.0000]
2023-04-19 09:33:12 [INFO] Quantize the model with default config.
2023-04-19 09:33:13 [INFO] Fx trace of the entire model failed, We will conduct auto quantization
2023-04-19 09:33:20 [INFO] |******Mixed Precision Statistics******|
2023-04-19 09:33:20 [INFO] +-----------------+----------+---------+
2023-04-19 09:33:20 [INFO] |     Op Type     |  Total   |   INT8  |
2023-04-19 09:33:20 [INFO] +-----------------+

CPU times: user 20.2 s, sys: 1.36 s, total: 21.5 s
Wall time: 10.6 s


In [54]:
# speed tests
print("mem quant:", model_iq.get_memory_footprint()/(1024*1024))   # 488MB
%time print(tokenizer.decode(model_iq.generate(input_ids, max_new_tokens=120, do_sample=False)[0]))

print()
print("mem regul:", model.get_memory_footprint()/(1024*1024))  # 1145MB
%time print(tokenizer.decode(model.generate(input_ids, max_new_tokens=120, do_sample=False)[0]))

# => HUH, IT SEEMS THE DYNAMIC QUANTIZED MODEL IS SLOWER
#   facinating outputs are slightly different with same inputs

mem quant: 488.55859375
<pad> Apple CEO Tim Cook is in Mumbai to inaugurate the company's first store in India. The company is planning to boost its production in India. The South Asian country of 1.4 billion people is also the second-largest smartphone market in the world.</s>
CPU times: user 8.12 s, sys: 8.07 ms, total: 8.13 s
Wall time: 2.36 s

mem regul: 1145.03662109375
<pad> Apple CEO Tim Cook is in Mumbai to inaugurate the company's first retail store in India. The company is planning to boost its production in India. The company is planning to boost its production in India.</s>
CPU times: user 9.6 s, sys: 0 ns, total: 9.6 s
Wall time: 1.2 s
